In [1]:
import pandas as pd
import numpy as np
from tqdm import tqdm
pd.set_option('mode.chained_assignment', None)

In [2]:
from os import listdir
from os.path import isfile, join
from datetime import datetime
from dateutil.parser import parse

In [3]:
confirmed_global_file = 'C:/Users/terra/corona_data/csse_covid_19_data/csse_covid_19_time_series/time_series_covid19_confirmed_global.csv'
confirmed_states_file = 'C:/Users/terra/corona_data/csse_covid_19_data/csse_covid_19_time_series/time_series_covid19_confirmed_US.csv'

recovered_file = 'C:/Users/terra/corona_data/csse_covid_19_data/csse_covid_19_time_series/time_series_covid19_recovered_global.csv'

deaths_global_file = 'C:/Users/terra/corona_data/csse_covid_19_data/csse_covid_19_time_series/time_series_covid19_deaths_global.csv'
deaths_states_file = 'C:/Users/terra/corona_data/csse_covid_19_data/csse_covid_19_time_series/time_series_covid19_deaths_US.csv'

In [4]:
confirmed_global_df = pd.read_csv(confirmed_global_file)
confirmed_states_df = pd.read_csv(confirmed_states_file)
recovered_df = pd.read_csv(recovered_file)
deaths_states_df = pd.read_csv(deaths_states_file)
deaths_global_df = pd.read_csv(deaths_global_file)
confirmed_global_df.head()

,Province/State,Country/Region,Lat,Long,1/22/20,1/23/20,1/24/20,1/25/20,1/26/20,1/27/20,...,3/27/20,3/28/20,3/29/20,3/30/20,3/31/20,4/1/20,4/2/20,4/3/20,4/4/20,4/5/20
0,NaN,Afghanistan,33.0000,65.0000,0,0,0,0,0,0,...,110,110,120,170,174,237,273,281,299,349
1,NaN,Albania,41.1533,20.1683,0,0,0,0,0,0,...,186,197,212,223,243,259,277,304,333,361
2,NaN,Algeria,28.0339,1.6596,0,0,0,0,0,0,...,409,454,511,584,716,847,986,1171,1251,1320
3,NaN,Andorra,42.5063,1.5218,0,0,0,0,0,0,...,267,308,334,370,376,390,428,439,466,501
4,NaN,Angola,-11.2027,17.8739,0,0,0,0,0,0,...,4,5,7,7,7,8,8,8,10,14


#### Remove US from Global Data

In [5]:
confirmed_global_df = confirmed_global_df[confirmed_global_df['Country/Region'] != 'US']
deaths_global_df = deaths_global_df[deaths_global_df['Country/Region'] != "US"]

#### Get list of dates

In [6]:
dates = confirmed_global_df.columns[4:]
dates

Index(['1/22/20', '1/23/20', '1/24/20', '1/25/20', '1/26/20', '1/27/20',
       '1/28/20', '1/29/20', '1/30/20', '1/31/20', '2/1/20', '2/2/20',
       '2/3/20', '2/4/20', '2/5/20', '2/6/20', '2/7/20', '2/8/20', '2/9/20',
       '2/10/20', '2/11/20', '2/12/20', '2/13/20', '2/14/20', '2/15/20',
       '2/16/20', '2/17/20', '2/18/20', '2/19/20', '2/20/20', '2/21/20',
       '2/22/20', '2/23/20', '2/24/20', '2/25/20', '2/26/20', '2/27/20',
       '2/28/20', '2/29/20', '3/1/20', '3/2/20', '3/3/20', '3/4/20', '3/5/20',
       '3/6/20', '3/7/20', '3/8/20', '3/9/20', '3/10/20', '3/11/20', '3/12/20',
       '3/13/20', '3/14/20', '3/15/20', '3/16/20', '3/17/20', '3/18/20',
       '3/19/20', '3/20/20', '3/21/20', '3/22/20', '3/23/20', '3/24/20',
       '3/25/20', '3/26/20', '3/27/20', '3/28/20', '3/29/20', '3/30/20',
       '3/31/20', '4/1/20', '4/2/20', '4/3/20', '4/4/20', '4/5/20'],
      dtype='object')

#### Replace NaN Values with Country Region if Province/State or Admin2 is empty

In [7]:
### Remove NaN values from Province/State Column
confirmed_global_df.loc[confirmed_global_df['Province/State'].isnull(),'Province/State'] = confirmed_global_df['Country/Region']
confirmed_states_df.loc[confirmed_states_df['Province_State'].isnull(),'Province_State'] = confirmed_states_df['Country_Region']
confirmed_states_df.loc[confirmed_states_df['Admin2'].isnull(),'Admin2'] = confirmed_states_df['Province_State']

deaths_global_df.loc[deaths_global_df['Province/State'].isnull(),'Province/State'] = deaths_global_df['Country/Region']
deaths_states_df.loc[deaths_states_df['Province_State'].isnull(),'Province_State'] = deaths_states_df['Country_Region']
deaths_states_df.loc[deaths_states_df['Admin2'].isnull(),'Admin2'] = deaths_states_df['Province_State']

recovered_df.loc[recovered_df['Province/State'].isnull(),'Province/State'] = recovered_df['Country/Region']

#### Remove '.' at end of locations/regions/states

In [8]:

confirmed_global_df['Province/State'] = confirmed_global_df['Province/State'].str.replace('.','')
confirmed_states_df['Province_State'] = confirmed_states_df['Province_State'].str.replace('.','')
confirmed_states_df['Admin2'] = confirmed_states_df['Admin2'].str.replace('.','')

deaths_global_df['Province/State'] = deaths_global_df['Province/State'].str.replace('.','')
deaths_states_df['Province_State'] = deaths_states_df['Province_State'].str.replace('.','')
deaths_states_df['Admin2'] = deaths_states_df['Admin2'].str.replace('.','')

recovered_df['Province/State'] = recovered_df['Province/State'].str.replace('.','')


### Post Object

Holds a list of documents that will be posted to the database

In [12]:
post = {'data': {}}

for x in dates:
    
    post['data'][x] = []
    


### STATES DATA

In [14]:
for x in tqdm(dates):
    for index, row in confirmed_states_df.iterrows():
        date = x
        fips = row['FIPS']
        location = row['Admin2']
        region = row['Country_Region']
        state = row['Province_State']
        lat = row['Lat']
        lng = row['Long_']
        confirmed = row[x]
        if confirmed < 1:
            continue
        else:
            obj = { 
                    'date': x,
                    'location': location,
                    'fips': fips,
                    'state': state,
                    'region': region,
                    'lat': lat,
                    'lng': lng,
                    'confirmed': int(confirmed)
                }
            post['data'][x].append(obj)
    for index, row in deaths_states_df.iterrows():
        date = x
        fips = row['FIPS']
        location = row['Admin2']
        deaths = row[x]
        
        for i in post['data'][x]:
            if i['fips'] == fips and i['location'] == location:
                i['deaths'] = deaths
                    


 89%|█████████████████████████████████████████████████████████████████████████▎        | 67/75 [00:46<00:09,  1.20s/it]


100%|██████████████████████████████████████████████████████████████████████████████████| 75/75 [00:57<00:00,  1.45s/it]

### GLOBAL DATA

In [15]:
for x in tqdm(dates):
    for index, row in confirmed_global_df.iterrows():
        date = x
        location = row['Province/State']
        region = row['Country/Region']
        lat = row['Lat']
        lng = row['Long']
        confirmed = row[x]
        if confirmed < 1:
            continue
        else:
            obj = { 
                'date': x,
                'location': location,
                'region': region,
                'lat': lat,
                'lng': lng,
                'confirmed': int(confirmed)
            }
            post['data'][x].append(obj)
    
    for index, row in deaths_global_df.iterrows():
        date = x
        location = row['Province/State']
        region = row['Country/Region']
        deaths = row[x]
        
        for i in post['data'][x]:
            if i['location'] == location and i['region'] == region:
                i['deaths'] = deaths
                
    for index, row in recovered_df.iterrows():
        date = x
        location = row['Province/State']
        region = row['Country/Region']
        recovered = row[x]
        
        for i in post['data'][x]:
            if i['location'] == location and i['region'] == region:
                i['recovered'] = recovered               


100%|██████████████████████████████████████████████████████████████████████████████████| 75/75 [00:06<00:00,  5.85it/s]

#### Add 0 to entries without recovered or death data

In [17]:
for x in post['data']:

    for z in post['data'][x]:

        try:
            if z['recovered']:
                pass
        except:
            z['recovered'] = 0
        
        try:
            if z['deaths']:
                pass
        except:
            z['deaths'] = 0

In [19]:
import pymongo
import dns
username = ''
password = ''

conn = 'mongodb+srv://' + username +':' + password + '@cluster0-paegd.mongodb.net/test?retryWrites=true&w=majority'
client = pymongo.MongoClient(conn)
db = client['corona_virus']
collection = db['daily_cases']

In [20]:
for x in tqdm(post['data']):
    for z in post['data'][x]:
        id = collection.insert_one(z).inserted_id


 89%|█████████████████████████████████████████████████████████████████████████▎        | 67/75 [17:26<10:27, 78.44s/it]


100%|█████████████████████████████████████████████████████████████████████████████████| 75/75 [31:37<00:00, 109.91s/it]